In [1]:
!pip install numpy
!pip install matplotlib
!pip install PIL


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from matplotlib.image import imread
import numpy as np
from PIL import Image

In [3]:
image = imread('data/img_V2.png')[:, :, 0:3]
image.shape

(690, 500, 3)

In [4]:
# convert image to 8 bit per channel
image = (image * 255).astype(np.uint8)

image

array([[[243, 205,  96],
        [192, 152,  31],
        [201, 156,  11],
        ...,
        [253, 253, 253],
        [255, 255, 255],
        [233, 233, 233]],

       [[242, 204,  95],
        [191, 151,  30],
        [200, 155,  10],
        ...,
        [253, 253, 253],
        [255, 255, 255],
        [229, 229, 229]],

       [[242, 204,  95],
        [191, 151,  30],
        [200, 155,  10],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [227, 227, 227]],

       ...,

       [[255, 255, 157],
        [237, 234,  95],
        [235, 234,  32],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 154],
        [233, 232,  90],
        [238, 239,  39],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 150],
        [232, 231,  87],
        [237, 239,  41],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [5]:
with open('output/encoded_image.BMP', 'wb') as f:
    # encode BMP image
    Image.fromarray(image).save(f, 'BMP')

In [6]:
code_encode = "5739 voilà le code secret"
# convert code to binary string forcing it to be 8 bits long
caracters_values = [ord(c) for c in code_encode]
code_string = ''.join([format(c, '08b') for c in caracters_values])
print(code_string)
code_string =format(255, '08b') + code_string +  format(4, '08b')
code_string

00110101001101110011001100111001001000000111011001101111011010010110110011100000001000000110110001100101001000000110001101101111011001000110010100100000011100110110010101100011011100100110010101110100


'111111110011010100110111001100110011100100100000011101100110111101101001011011001110000000100000011011000110010100100000011000110110111101100100011001010010000001110011011001010110001101110010011001010111010000000100'

In [7]:
offset = 100000
code_string = "".join(["0" for i in range(offset)]) + code_string

In [8]:
encoding_length = image.shape[0] * image.shape[1] * image.shape[2]
print("encoding space 3 channels ", encoding_length)
print("encoding space 1 channel ", encoding_length // image.shape[2])
print("code length ", len(code_string))
assert len(code_string) <= encoding_length // image.shape[2]

encoding space 3 channels  1035000
encoding space 1 channel  345000
code length  100216


In [9]:
def encode_bit(bit, pixel_value):
    if bit == 1:
        if pixel_value % 2 == 0:
            return pixel_value + 1
        elif pixel_value == 255:
            return pixel_value - 1
        else:
            return pixel_value
    else:
        if pixel_value % 2 == 0:
            return pixel_value
        elif pixel_value == 0:
            return pixel_value + 1
        else:
            return pixel_value - 1


In [10]:
# for i, bit in enumerate(code_string):
#     pixel = image[i% image.shape[1], i//image.shape[1] , 0]
#     print(pixel, bit, encode_bit(int(bit), pixel))

In [11]:

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if i * image.shape[1] + j < len(code_string):
            #for k in range(image.shape[2]):
            k = 0
            image[i, j, k] = encode_bit(int(code_string[i * image.shape[1] + j]), image[i, j, k])
        else:
            # for k in range(image.shape[2]):
            k=0
            image[i, j, k] = encode_bit(0, image[i, j, k])

with open('output/encoded_image.bmp', 'wb') as f:
    # encode BMP image
    Image.fromarray(image).save(f, 'BMP')

In [12]:
import re
def decode_bit(pixel_value):
    return pixel_value % 2


def decode_image(image):
    k = 0
    code_string = ''
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            code_string += str(decode_bit(image[i, j, k]))
    return code_string

def to_string(decoded_bits):
    caracters_values = [decoded_bits[i:i + 8] for i in range(0, len(decoded_bits), 8)]
    return ''.join([chr(int(c, 2)) for c in caracters_values])
    

In [13]:
decoded_bits = decode_image(image)
decoded_string = to_string(decoded_bits)
decoded_string.strip("\x00"), len(decoded_string)

('ÿ5739 voilà le code secret\x04', 43125)

In [14]:
code_string

'...'

In [15]:
loaded_image = imread('output/encoded_image.bmp')
loaded_image.shape
loaded_image

array([[[242, 205,  96],
        [192, 152,  31],
        [200, 156,  11],
        ...,
        [252, 253, 253],
        [254, 255, 255],
        [232, 233, 233]],

       [[242, 204,  95],
        [190, 151,  30],
        [200, 155,  10],
        ...,
        [252, 253, 253],
        [254, 255, 255],
        [228, 229, 229]],

       [[242, 204,  95],
        [190, 151,  30],
        [200, 155,  10],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [226, 227, 227]],

       ...,

       [[254, 255, 157],
        [236, 234,  95],
        [234, 234,  32],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [254, 255, 255]],

       [[254, 255, 154],
        [232, 232,  90],
        [238, 239,  39],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [254, 255, 255]],

       [[254, 255, 150],
        [232, 231,  87],
        [236, 239,  41],
        ...,
        [254, 255, 255],
        [254, 255, 255],
        [254, 255, 255]]

In [16]:
decoded_bits = decode_image(image)
decoded_string = to_string(decoded_bits)
decoded_string.strip("\x00"), len(decoded_string)

('ÿ5739 voilà le code secret\x04', 43125)